In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('datasets/FHV_Base_Aggregate_Report.csv')
df.head()

,Base License Number,Base Name,DBA,Year,Month,Month Name,Total Dispatched Trips,Total Dispatched Shared Trips,Unique Dispatched Vehicles
0,B01236,BLS LIMO SVC OF NY INC,NaN,2018,5,May,3413,0,143
1,B01285,UNION RADIO DISPATCH INC,NaN,2020,1,January,9063,0,53
2,B02892,SEAMAN EXPRESS CORP.,NaN,2018,2,February,39,0,8
3,B02429,SEAGULL RIDE CORP,NaN,2021,7,July,3592,0,11
4,B00552,HINO-MARU ENTERPRISES INC.,NaN,2016,5,May,974,0,50


In [3]:
df.shape

(43045, 9)

In [4]:
df['Base License Number'].value_counts()

Base License Number
UBER      90
B00381    89
B00254    89
B00477    89
B00225    88
          ..
B03220     1
B01222     1
B03230     1
B03227     1
b02725     1
Name: count, Length: 1009, dtype: int64

In [5]:
df['Unique Dispatched Vehicles'].sort_values(ascending=False)

962      79178
37435    78973
37874    78257
39851    78093
40622    77712
         ...  
40575        1
12615        1
7294         1
30589        1
15419        1
Name: Unique Dispatched Vehicles, Length: 43045, dtype: int64

In [6]:
fhvhv = pd.read_parquet('../fhvhv_tripdata_2023-04.parquet', engine='pyarrow')

更改column name，方便稍後join

In [7]:
fhvhv.rename(columns={'dispatching_base_num': 'Base License Number'}, inplace=True)

In [9]:
anti = df.merge(fhvhv, on='Base License Number', how='left', indicator=True)
anti.head()

,Base License Number,Base Name,DBA,Year,Month,Month Name,Total Dispatched Trips,Total Dispatched Shared Trips,Unique Dispatched Vehicles,hvfhs_license_num,...,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,_merge
0,B01236,BLS LIMO SVC OF NY INC,NaN,2018,5,May,3413,0,143,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,B01285,UNION RADIO DISPATCH INC,NaN,2020,1,January,9063,0,53,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,B02892,SEAMAN EXPRESS CORP.,NaN,2018,2,February,39,0,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,B02429,SEAGULL RIDE CORP,NaN,2021,7,July,3592,0,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,B00552,HINO-MARU ENTERPRISES INC.,NaN,2016,5,May,974,0,50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [10]:
filter = anti.loc[anti['_merge']=='left_only', 'Base License Number']
filter.head()

0    B01236
1    B01285
2    B02892
3    B02429
4    B00552
Name: Base License Number, dtype: object

過濾，結果代表所有的base中，2023年4月有行程記錄的base

In [12]:
result = df[df['Base License Number'].isin(filter)]
result

,Base License Number,Base Name,DBA,Year,Month,Month Name,Total Dispatched Trips,Total Dispatched Shared Trips,Unique Dispatched Vehicles
0,B01236,BLS LIMO SVC OF NY INC,NaN,2018,5,May,3413,0,143
1,B01285,UNION RADIO DISPATCH INC,NaN,2020,1,January,9063,0,53
2,B02892,SEAMAN EXPRESS CORP.,NaN,2018,2,February,39,0,8
3,B02429,SEAGULL RIDE CORP,NaN,2021,7,July,3592,0,11
4,B00552,HINO-MARU ENTERPRISES INC.,NaN,2016,5,May,974,0,50
...,...,...,...,...,...,...,...,...,...
43040,B01233,NEW LACONIA RAD DISP INC,NaN,2016,8,August,22274,0,90
43041,B01318,LOWER EAST SIDE C/S CORP,NaN,2019,8,August,2152,0,21
43042,B01065,EL BARRIO'S CAR SERVICE LLC,NaN,2019,8,August,8868,0,63
43043,B02970,FLYTE LINE TRANSPORTATION LLC,ADDISON LEE,2017,8,August,1241,0,90


再搭配current_bases.csv的資料，應可以找出?